- 保监会 分类模型 2 训练 

# 基本设置

In [1]:
##load packages, needed
# encoding=utf-8

import jieba
import sys
import re
import time
import string
from sklearn import feature_extraction
from sklearn.pipeline import Pipeline,FeatureUnion

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2,mutual_info_classif,f_classif 

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import metrics

from sklearn.base import BaseEstimator, TransformerMixin
from collections import defaultdict

import pickle

import joblib
%matplotlib inline
import numpy as np
import pandas as pd
import pre_cor
import os
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine
from pandas.io import sql
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [2]:
class StatsFeatures(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.neg = set()
        f = open("corpus/neg_words.txt","r+", encoding='UTF-8')
        for content in f:
            self.neg.add(content)
        f.close()

    def fit(self, X, y=None):
        return self

    def getcnt(self,x):        
        return len(list(set(x)))

    def getnegcnt(self,x):
        negcnt = 0
        words = x.split()
        for w in words:
            if w in self.neg:
                negcnt = negcnt+1
        return negcnt
    
    def transform(self, X):
        data = []
        for x in X:
            if len(x) == 0:
                length  = 1
            else :
                length = len(x)
            data.append([len(x),self.getcnt(x),self.getcnt(x)/length,
                         self.getnegcnt(x),self.getnegcnt(x)/length])            
        return data

## 上一版模型读入

In [3]:
# 上一版模型
# from sklearn.externals import joblib
# pipeline_old = joblib.load( "model/circ_cor_0625.pkl.z") 

# 导入数据

In [4]:
title = []
filename = 'data/titles.txt'
fid = open(filename, "r+", encoding='UTF-8')
for f in fid:
    title.append(f.strip().replace('\n', ''))
fid.close()
print(len(title))

7511


In [5]:
content = []
filename = 'data/contents.txt'
fid = open(filename, "r+", encoding='UTF-8')
for f in fid:
    content.append(f.strip().replace('\n', ''))
fid.close()
print(len(content))
# content[:2]

7511


In [6]:
title_content = [t + ' ' + c for t,c in zip(title, content)]
print(len(title_content))

7511


In [13]:
# del generate_keywords
from toolkits.nlp import generate_keywords

In [10]:
g_k = generate_keywords.generate_keywords(topk = 200)
g_k.gen_keywords(title_content[:2000])
# g_k.gen_keywords_by_class(title_content, label, topK=20, v_thre=5)


['保险', '公司', '保险公司', '产品', '进行', '业务', '投资', '分公司', '监管', '销售']
keywords num:  200


['保险',
 '公司',
 '保险公司',
 '产品',
 '进行',
 '业务',
 '投资',
 '分公司',
 '监管',
 '销售',
 '服务',
 '风险',
 '机构',
 '相关',
 '中国',
 '管理',
 '图片',
 '批复',
 '工作',
 '保障',
 '规定',
 '市场',
 '决定',
 '客户',
 '有限公司',
 '保费',
 '问题',
 '资格',
 '任职',
 '支公司',
 '发展',
 '企业',
 '消费者',
 '保单',
 '理赔',
 '要求',
 '罚款',
 '投保',
 '处罚',
 '经营',
 '购买',
 '行业',
 '行为',
 '应当',
 '人员',
 '提供',
 '资金',
 '作出',
 '之日起',
 '金融',
 '行政处罚',
 '开展',
 '寿险',
 '经济',
 '计划',
 '董事',
 '任命',
 '核准',
 '信息',
 '中心',
 '股权',
 '收到',
 '通知',
 '保险业',
 '设立',
 '数据',
 '投保人',
 '银行',
 '车险',
 '方式',
 '公告',
 '活动',
 '监事',
 '责任',
 '股份',
 '情况',
 '请示',
 '资产',
 '需要',
 '保险产品',
 '保险资金',
 '项目',
 '时间',
 '当事人',
 '决定书',
 '基金',
 '保险代理',
 '保险法',
 '营业',
 '代理',
 '医疗保险',
 '办理',
 '医疗',
 '依法',
 '申请',
 '网站',
 '内容',
 '平台',
 '保险业务',
 '人身保险',
 '专业',
 '没有',
 '费用',
 '价值',
 '资料',
 '增长',
 '发生',
 '负责人',
 '社会',
 '方面',
 '可能',
 '股东',
 '变更',
 '乱象',
 '电话',
 '收益',
 '集团',
 '管理人员',
 '选择',
 '营销',
 '事实',
 '实体',
 '存在',
 '利用',
 '调查',
 '科技',
 '被保险人',
 '分红',
 '地址',
 '创新',
 '是否',
 '不得',
 '文件',
 '审核',
 '保险条款',
 '总经理',
 '表示',
 '出现',

In [14]:
g_k.save_txt('keywords_i.txt')

In [17]:
g_k.gen_keywords_by_class(title_content[:3000], label[:3000], v_thre=3)

['产品销售' '公司内部管理' '其他相关报道' '噪音' '消费服务' '监管' '行业' '资本市场']
依据类别生成关键词：
['价值', '停售', '支行', '打折', '资产', '车辆', '癌症', '需求', '业务', '总额']
keywords num:  419
类别：产品销售 -- 文本个数：253，label个数:253，取词个数：419
['成都', '扬州', '价值', '总裁', '支行', '电销', '意识', '资产', '联想', '公网']
keywords num:  524
类别：公司内部管理 -- 文本个数：417，label个数:417，取词个数：524
['价值', '区长', '支行', '透过', '本站', '带领', '文章', '种植', '爱心', '身材']
keywords num:  529
类别：其他相关报道 -- 文本个数：426，label个数:426，取词个数：529
['养殖', '价值', '晋中市', '熟手', '基点', '制造', '空间', '座谈会', '意识', '资产']
keywords num:  1242
类别：噪音 -- 文本个数：665，label个数:665，取词个数：1242
['养殖', '当事人', '人员', '经营', '价值', '年缴', '定损', '理赔金', '我司', '湖南']
keywords num:  292
类别：消费服务 -- 文本个数：156，label个数:156，取词个数：292
['扬州', '价值', '制造', '口径', '延误', '资产', '董事会', '文章', '保险代理', '中间业务']
keywords num:  538
类别：监管 -- 文本个数：644，label个数:644，取词个数：538
['备案', '养殖', '经营', '价值', '人员', '壽險', '股权', '贸易', '外贸', '提供']
keywords num:  297
类别：行业 -- 文本个数：140，label个数:140，取词个数：297
['有变', '价值', '受访', '资产', '拉升', '选股', '反弹', '保险代理', '乐视', '整理']
keywords num: 

In [18]:
print(g_k.keywords)

['晋中市', '透过', '座谈会', '延误', '优化', '董事会', '联想', '拟任', '选股', '校园', '乐视', '需求', '表达', '折价', '不计', '接力', '蚯蚓', '意外事故', '排行', '增幅', '矿业', '抽查', '活跃', '情系', '帖子', '加大', '禁止', '包含', '成都市', '强制保险', '贵州', '电子', '局长', '提交', '中宏', '西安', '低估', '大额', '头版', '职业病', '电缆', '社会化', '保通', '黔南', '标的', '负担', '雅砻江', '不用', '民宿', '车网', '车损险', '遭遇', '监测', '经历', '关系', '受托', '远海', '自查', '放心', '河北省', '身体', '负责', '理性', '上行', '厂里', '金牌榜', '动车', '看看', '香氛', '标段', '章程', '保险费用', '贫困人口', '首例', '登陆', '撤诉', '继承人', '视为', '自燃', '华夏', '志愿', '遇上', '糖尿病', '信用卡', '低息', '循化', '抚养人', '商旅', '倒闭', '填写', '新能源', '保險', '原油', '责任编辑', '盘面', '地产股', '天标', '白对虾', '消防', '同事', '主管', '安徽', '版图', '芙蓉', '沪深股市', '犯愁', '党员干部', '兴安盟', '车界', '信访', '意外保险', '拆完', '下降', '实施办法', '空调', '尿素', '访问', '获得者', '最佳答案', '回报', '学员', '全流通', '责令', '乘客', '善款', '质询', '出门', '不想', '干支', '保單', '模拟', '公共关系', '前提', '观察期', '协商', '白血病', '混合', '合约', '专员办', '抓住', '外汇', '损失', '灾情', '净流入', '唐山市', '客戶', '灌南', '行政许可', '武川县', '盘升', '净资产', '并购', '获奖', '待遇', '反洗钱法', '金融界', '妻子', '潮头

In [8]:
label_dic={'监管':1,'行业':2,'产品销售':3,'资本市场':4,'公司内部管理':5,'消费服务':6,'其他相关报道':7,'噪音':8}
class_name_dict = {v: k for k, v in label_dic.items()}
class_name_dict

{1: '监管',
 2: '行业',
 3: '产品销售',
 4: '资本市场',
 5: '公司内部管理',
 6: '消费服务',
 7: '其他相关报道',
 8: '噪音'}

In [8]:
label = []
filename = 'data/labels.txt'
fid = open(filename, "r+", encoding='UTF-8')
for f in fid:
    label.append(f.strip().replace('\n', ''))
fid.close()
print(len(label))
label[:30]

7511


['噪音',
 '噪音',
 '产品销售',
 '噪音',
 '消费服务',
 '噪音',
 '噪音',
 '资本市场',
 '监管',
 '公司内部管理',
 '资本市场',
 '其他相关报道',
 '噪音',
 '噪音',
 '噪音',
 '噪音',
 '消费服务',
 '噪音',
 '噪音',
 '资本市场',
 '监管',
 '监管',
 '行业',
 '行业',
 '噪音',
 '噪音',
 '噪音',
 '噪音',
 '公司内部管理',
 '产品销售']

# 训练模型

## 分割训练集、测试集

In [12]:
X_train, X_test, y_train, y_test = train_test_split(title_content, label, test_size=0.3, random_state=42)
print('训练集：',len(y_train))
print('训练集-各类数量：',Counter(y_train))
print('测试集：',len(y_test))
print('测试集-各类数量：',Counter(y_test))

训练集： 5257
训练集-各类数量： Counter({'噪音': 2376, '监管': 1034, '其他相关报道': 472, '公司内部管理': 399, '资本市场': 329, '产品销售': 231, '行业': 218, '消费服务': 198})
测试集： 2254
测试集-各类数量： Counter({'噪音': 1005, '监管': 456, '其他相关报道': 207, '公司内部管理': 181, '资本市场': 132, '行业': 100, '产品销售': 91, '消费服务': 82})


## xgboost

### train

In [ ]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tf_idf', Pipeline([
            ('counts', CountVectorizer()),
            ('tf_idf', TfidfTransformer()),
            ('chi', SelectKBest(chi2, k=20000))
        ])),
        ('len_stats', StatsFeatures())
    ])),
    ('classifier', XGBClassifier(max_depth=7,objective='multi:softmax', num_class=2))
])

In [ ]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tf_idf', Pipeline([
            ('counts', CountVectorizer()),
            ('tf_idf', TfidfTransformer()),
            ('chi', SelectKBest(chi2, k=20000))
        ])),
        ('len_stats', StatsFeatures())
    ])),
    ('classifier', XGBClassifier(max_depth=7,objective='multi:softmax', num_class=2))
])

pipeline.fit(X_train, y_train)

In [ ]:
print(pipeline.score(X_train, y_train))

### test

In [ ]:
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

In [ ]:
# 上一版模型 
y_pred_class = pipeline_old.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

# 模型保存

In [ ]:
from sklearn.externals import joblib
joblib.dump(pipeline, "model/circ_cor_0703.pkl.z")  

# 保存本文件

In [ ]:
if 0:
    import datetime as dt
    
    def output_HTML(read_file, output_file):
        from nbconvert import HTMLExporter
        import codecs
        import nbformat
        exporter = HTMLExporter()
        # read_file is '.ipynb', output_file is '.html'
        output_notebook = nbformat.read(read_file, as_version=4)
        output, resources = exporter.from_notebook_node(output_notebook)
        codecs.open(output_file, 'w', encoding='utf-8').write(output)

    html_file_folder = 'html_files'
    if not os.path.exists(html_file_folder):
        os.makedirs(html_file_folder)

    today = dt.datetime.now().strftime('%Y%m%d')
    current_file = 'circ_cor_model_2_train.ipynb'
    output_file = 'html_files\%s_%s.html'%(os.path.splitext(current_file)[0], today)
    output_HTML(current_file, output_file)